In [1]:
import time
import numpy as np
import sys
import requests
import json
from random import sample
import random

In [2]:
def delay_print(s):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(0.05)

In [3]:
def api_call(url):
    """Makes api call to the Pokemon type endpoint and returns json text"""
    response = requests.get(url)
    response_json = json.loads(response.text)
    return response_json


In [4]:
class Pokemon:
    def __init__(self, name, types, moves, speed, health=100):
        self.name = name
        self.types = types
        self.moves = moves
        self.health = health
        self.bars = 100 # JS changed health to 100
        self.speed = speed
        self.moves_list = []
        self.pokemon_url = 'https://pokeapi.co/api/v2/pokemon/'+self.name+'/'
        self.weaknesses = []
        self.resistances = []
        self.types_list = []


    def get_moves(self): 
        """Creates a list of moves for battle. The moves are random
        assignments from all available moves in the api"""
        response_json = api_call(self.pokemon_url)
        num_of_moves = len(response_json['moves'])
        move_index = sample(range(num_of_moves), 4) #randomly selects 4 out of all moves available

        for i in move_index:
            self.moves_list.append(
                response_json['moves'][i]['move']['name'])

        self.moves_list = np.unique(np.array(self.moves_list)).tolist() #assigns random moves to a list for this pokemon
        self.moves = np.unique(np.array(self.moves_list)).tolist() #resets original moves to random list


    def poke_types(self):
        """Returns a list of the Pokemon's type(s)"""
        self.types_list.clear() #empty the list so that duplicates are not created if ran multiple times
        response_json = api_call(self.pokemon_url)
        for i in range(len(response_json['types'])):
            type_resp = response_json['types'][i]['type']['name']
            self.types_list.append(type_resp)


    

    def move_factors(self):
        """Finds the weaknesses and resistance types for this Pokemon"""
        for i in self.poke_types():
            type_url = 'https://pokeapi.co/api/v2/type/'+i+'/'
            type_response_json = api_call(type_url)

            for j in range(len(type_response_json['damage_relations']['double_damage_from'])):
                self.weaknesses.append(type_response_json['damage_relations']['double_damage_from'][j]['name'])

            for k in range(len(type_response_json['damage_relations']['half_damage_from'])):
                self.resistances.append(type_response_json['damage_relations']['half_damage_from'][k]['name'])

            self.weaknesses = np.unique(np.array(self.weaknesses)).tolist()
            self.resistances = np.unique(np.array(self.resistances)).tolist()

    
def move_info(move): 
    move = move.replace(' ', '-').lower()
    url = 'https://pokeapi.co/api/v2/move/'+move+'/'
    response_json = api_call(url)

    move_type = response_json['type']['name']
    # move_power = response_json['power']

    return move_type, # move_power

    
def damage_multiplier(defending_mon, move): 
    d_resistences = set(defending_mon.resistances)
    d_weaknesses = set(defending_mon.weaknesses)
    m_type = move_info(move)
    move_types = set(m_type)
    rando = random.randint(1,10)
    
    if rando == 7: # Adding some fun uncertinity to the match
        multiplier = 0
    elif rando == 3:
        multiplier = 30
    elif move_types.intersection(d_weaknesses):
        multiplier = 10 # JS increased value
    elif move_types.intersection(d_resistences):
        multiplier = .5
    else: 
        multiplier = 3 # JS increased value
    
    return multiplier




In [5]:
def battle_turn1(quicker_mon, slower_mon):
    print("Go", quicker_mon.name, "!")
    
    for i, x in enumerate(quicker_mon.moves):
        print(i+1, x)
    index = int(input("Pick a move: "))
    
    print(quicker_mon.name ,"used", list(quicker_mon.moves.keys())[index-1])
    time.sleep(1)
    print(quicker_mon.name, "did", list(quicker_mon.moves.values())[index-1]*damage_multiplier(slower_mon,list(quicker_mon.moves.keys())[index-1]), "damage!")
    #print('[multipler = ', damage_multipler, ']')

    slower_mon.bars -= list(quicker_mon.moves.values())[index-1]*damage_multiplier(slower_mon,list(quicker_mon.moves.keys())[index-1])
    slower_mon.health = ""

    for i in range(int(slower_mon.bars)):
        slower_mon.health += "="

    print(quicker_mon.name ,"health:", quicker_mon.bars) # JS changed from == to return numeric value
    print(slower_mon.name ,"health:", slower_mon.bars, '\n') # JS changed from == to return numeric value
    print('     ') # Adding separator

In [6]:
def battle_turn2(quicker_mon, slower_mon):
    print("Go", slower_mon.name, "!")
    for i, x in enumerate(slower_mon.moves):
        print(i+1, x)

    index = int(input("Pick a move: "))
    
    print(slower_mon.name ,"used", list(slower_mon.moves.keys())[index-1])
    time.sleep(1)
    print(slower_mon.name, "did", list(slower_mon.moves.values())[index-1]*damage_multiplier(quicker_mon,list(slower_mon.moves.keys())[index-1]), "damage!")
    #print('[multipler = ', damage_multipler, ']')


    quicker_mon.bars -= list(slower_mon.moves.values())[index-1]*damage_multiplier(quicker_mon,list(slower_mon.moves.keys())[index-1])
    quicker_mon.health = ""

    for i in range(int(quicker_mon.bars)):
        quicker_mon.health += "="
        
    print(slower_mon.name ,"health:", slower_mon.bars) # JS changed from == to return numeric value
    print(quicker_mon.name ,"health:", quicker_mon.bars)  # JS changed from == to return numeric value          
    print('     ')

In [7]:
def battle(Pokemon1, Pokemon2): 
    
    print("-----POKEMON BATTLE-----")
    print("Pokemon 1:", Pokemon1.name)
    print("TYPE/", Pokemon1.types)
    print("\nVS\n")
    print("Pokemon 2:", Pokemon2.name)
    print("TYPE/", Pokemon2.types, '\n')
    
#evaluate speed of pokemon
    
    while Pokemon1.speed > Pokemon2.speed == True:
        quicker_mon = Pokemon1
        slower_mon = Pokemon2
    
    else: 
        slower_mon = Pokemon1
        quicker_mon = Pokemon2
        
    while (quicker_mon.bars > 0) and (slower_mon.bars > 0):
            
            battle_turn1(quicker_mon, slower_mon)
            
            if slower_mon.bars <= 0:
                delay_print("\n..." + slower_mon.name + ' fainted.')
                break
                
 #mirroring the attack pattern of the quicker mon but for the slower mon

            battle_turn2(quicker_mon, slower_mon)           
            
            if quicker_mon.bars <= 0:
                delay_print("\n..." + quicker_mon.name + ' fainted.')
                break

In [8]:
if __name__ == '__main__':
    gyarados = Pokemon('gyarados', 'water', {'crunch': 1, 'surf': 3}, 4)
    aerodactyl = Pokemon('aerodactyl', 'rock', {'air-cutter': 2, 'head-smash': 3}, 7)
    
#could use api calls for move type and move power instead of using dictionaries
    #for damage calcs, could use power % 10 to simplify health ga

In [ ]:
battle(gyarados, aerodactyl)

-----POKEMON BATTLE-----
Pokemon 1: gyarados
TYPE/ water

VS

Pokemon 2: aerodactyl
TYPE/ rock 

Go aerodactyl !
1 air-cutter
2 head-smash
Pick a move: 1
aerodactyl used air-cutter
aerodactyl did 6 damage!
aerodactyl health: 100
gyarados health: 40 

     
Go gyarados !
1 crunch
2 surf
Pick a move: 2
gyarados used surf
gyarados did 9 damage!
gyarados health: 40
aerodactyl health: 91
     
Go aerodactyl !
1 air-cutter
2 head-smash
Pick a move: 1
aerodactyl used air-cutter
aerodactyl did 6 damage!
aerodactyl health: 91
gyarados health: 34 

     
Go gyarados !
1 crunch
2 surf
Pick a move: 2
gyarados used surf
gyarados did 9 damage!
gyarados health: 34
aerodactyl health: 82
     
Go aerodactyl !
1 air-cutter
2 head-smash


In [ ]:
gyarados.moves #original attritubtes 


In [ ]:
gyarados.get_moves() #updates moves to new choices but now has no damage associated 
gyarados.moves 

In [ ]:
gyarados.moves_list #just a list of the random moves. work in progress

In [ ]:
gyarados.types

In [ ]:
gyarados.poke_types()
gyarados.types_list

In [ ]:
ivysaur = Pokemon('ivysaur', 'grass', 'na', 4,)

In [ ]:
ivysaur.get_moves()

In [ ]:
ivysaur.poke_types()

In [ ]:
ivysaur.types_list

In [ ]:
ivysaur.moves